## Assignment 3
### GTECH 731
### Spring 2023
### Jake Markey


In [10]:
"""
Task 1 Read in a data file of all counties in the US.
Make a list of unique county names.
Note: There are no changes in this section between assignment 2 and 3
"""

import json, random

#Read the county json file to a dictionary variable
with open("gz_2010_us_050_00_5m.json", 'r') as f:
  data = json.load(f)

#confirm the dictionary loaded all of the data by printing number of features
print(f"The file contains {len(data['features'])} counties")

#make a list of county names
featureList = data['features']

#make a dictionary containing all counties and the frequency with which they appear
CountyDict = {}

for i in featureList: 
    name = i['properties']['NAME']
    if CountyDict.get(name) == None:
        CountyDict.update({name:1})
    else:
        current_count = CountyDict.get(name)
        updated_count = current_count + 1
        CountyDict.update({name : updated_count})

#print a random county name and its frequency
sample_county = random.choice(list(CountyDict.keys()))
print(f"Do you know how many counties are called {sample_county} County in the United States? Well let me tell you! The answer is {CountyDict[sample_county]}.")


The file contains 3221 counties
Do you know how many counties are called Grant County in the United States? Well let me tell you! The answer is 15.


In [4]:
"""
Task 2: Derive the numbers of counties that use these three names, respectively.For each of them, list their county name and state code. 
Note: using the JSON file with state FIPS codes I chose to print full state names. Also, this section now includes new functions and an option for user input, which
is an update from assignment 2.
"""

#Read the state/ fips json file to a dictionary variable
with open("fipsToState.json", 'r') as f:
    StateFIP_Dict = json.load(f)

"""
Create nested loops within a function. The first, outer loop will repeat the inner loop x times. The inner loop will check each county name against the preceding one(s), 
storing the highest value until another county knocks it out of its position. The winning county will be removed from the list and the process will be 
re-iterated- yieldeing the second and third place counties (and onwards depending on the user's input ). 

Assignment 3 Note: The purpose of the function and allowing user input is to demonstrate that it can be repurposed for other applications, although the variable names 
are still specific to the county application
"""
#create a list varaible containing only unique county names
county_names = list(CountyDict.keys())

def CommonProperties(length):
    #create an empty dictionary to store the 3 most common county names and the number they appear.
    FrequentProperties = {}

    #nested loops to find three most common county names. 
    for i in range(length):
        frequency = 0
        for county in county_names:
            if CountyDict.get(county) > frequency:
                frequency = CountyDict.get(county)
                popular_county_name = county
            else: 
                pass
        FrequentProperties.update({popular_county_name : frequency})
        county_names.remove(popular_county_name)
    return FrequentProperties   
    

#ask user to inut length of list (top x properties) and query the FrequentProperties Function
length = int(input("I would like to know the top x most common county names in the US. x = "))
FrequentProperties = CommonProperties(length)
print(f"This dictionary contains the {length} most common county names in the U.S. and the respective frequency: \n {FrequentProperties} \n")

#write the names of the states for the top x counties into a new dictionary
countyStates = {}
for prop in FrequentProperties: 
    TempStateList = []
    for feature in featureList: 
        if prop == feature['properties']['NAME']:
            TempStateList.append(StateFIP_Dict.get(feature['properties']['STATE']))
        else:
            pass
    countyStates.update({prop:TempStateList})
    
#print the results using a comprehension

print("\n".join([f"There are {FrequentProperties[prop]} {prop} Counties that can be found in the following states: {', '.join(map(str, countyStates[prop]))}.\n" for prop in FrequentProperties]))


This dictionary contains the 5 most common county names in the U.S. and the respective frequency: 
 {'Washington': 31, 'Franklin': 26, 'Jefferson': 26, 'Jackson': 24, 'Lincoln': 24} 

There are 31 Washington Counties that can be found in the following states: Colorado, Florida, Arkansas, Illinois, Iowa, Maine, Maryland, Missouri, Tennessee, Oklahoma, Oregon, Wisconsin, Utah, Vermont, Kansas, Indiana, Kentucky, Minnesota, Geogia, Alabama, Louisiana, Idaho, New York, North Carolina, Nebraska, Mississippi, Pennsylvania, Ohio, Rhode Island, Virginia, Texas.

There are 26 Franklin Counties that can be found in the following states: Geogia, Alabama, Indiana, Idaho, Kentucky, Iowa, Mississippi, New York, Texas, Arkansas, Florida, Louisiana, Maine, Massachusetts, Illinois, Kansas, Missouri, Tennessee, Nebraska, North Carolina, Pennsylvania, Ohio, Virginia, Virginia, Washington, Vermont.

There are 26 Jefferson Counties that can be found in the following states: Illinois, Geogia, Iowa, Oregon, 

In [11]:
"""
Task 3 Basic statistics by state

For each state, find

    The number of counties
    The name and size (census area) of the biggest and smallest county by area
    The total and average area of counties

Assignment 3 Note: The major update for this assignment was creating the DictionaryFactory function. 
"""

#create a dictionary with each state name as the key, and each value being a list of counties stored with unique IDs (GeoID)
StateCounty_Dict = {}
for state in list(StateFIP_Dict.keys()):
    countylist = []
    StateCounty_Dict.update({StateFIP_Dict.get(state):countylist})
    for i in featureList:
        if state == i["properties"]["STATE"]:
            countylist.append(i['properties']['GEO_ID'])
            StateCounty_Dict.update({StateFIP_Dict.get(state):countylist})
        else: 
            pass

#Create a function that writes dictionaries from the contents in the featureList variable
def DictionaryFactory(key, value):
    TempDictionary = {i['properties'][key]: i['properties'][value] for i in featureList}
    return TempDictionary

#create a dictionary of all county sizes
CountySize_Dict = DictionaryFactory('GEO_ID','CENSUSAREA')

#create a dictionary that matches GeoID to county name
GeoIDCountyName_Dict = DictionaryFactory('GEO_ID','NAME')


#funtion that identifies the largest and smallest counties in a state and prints the respective size (rounded to two decimals)
def BigSmallCounties(state):
    counties = StateCounty_Dict.get(state)
    biggest = counties[1]
    smallest = counties[1]
    for county in counties: 
        if CountySize_Dict[county] > CountySize_Dict[biggest]:
            biggest = county
        elif CountySize_Dict[county] < CountySize_Dict[smallest]:
            smallest = county
        else: 
            pass
    print(f"The largest county is {GeoIDCountyName_Dict.get(biggest)} at {CountySize_Dict.get(biggest):.2f} square miles and the smallest is {GeoIDCountyName_Dict.get(smallest)} at {CountySize_Dict.get(smallest):.2f} square miles.")

#function to compute the total and average county size in the chosen state
def TotalAverageCountySize(state):
    counties = StateCounty_Dict.get(state)
    TotalSize = 0
    for county in counties:
        TotalSize += CountySize_Dict[county]
    MeanSize = TotalSize / len(counties)
    print(f"Cobined, the counties in {state} are {TotalSize:.2f} square miles. The mean county size is {MeanSize:.2f} square miles.\n")

#loop that enables the user to learn about the counties in a state of his or her choice. Loop breaks when a string other than a state is entered. 
while True:
    states = StateCounty_Dict.keys()
    UserState = str(input("Which U.S. state would you like to know about? Type anything other than a state name to break"))
    if UserState in states: 
        print(f"There are {len(StateCounty_Dict.get(UserState))} counties in {UserState}.")
        BigSmallCounties(UserState)
        TotalAverageCountySize(UserState)
    else: 
        break




There are 64 counties in Colorado.
The largest county is Las Animas at 4772.67 square miles and the smallest is Broomfield at 33.03 square miles.
Cobined, the counties in Colorado are 103641.89 square miles. The mean county size is 1619.40 square miles.

There are 5 counties in Hawaii.
The largest county is Hawaii at 4028.42 square miles and the smallest is Kalawao at 11.99 square miles.
Cobined, the counties in Hawaii are 6422.63 square miles. The mean county size is 1284.53 square miles.

There are 21 counties in New Jersey.
The largest county is Burlington at 798.58 square miles and the smallest is Hudson at 46.19 square miles.
Cobined, the counties in New Jersey are 7354.22 square miles. The mean county size is 350.20 square miles.

There are 16 counties in Maine.
The largest county is Aroostook at 6671.33 square miles and the smallest is Sagadahoc at 253.69 square miles.
Cobined, the counties in Maine are 30842.92 square miles. The mean county size is 1927.68 square miles.

There 

Input for previous code was "Colorado" "Hawaii" "New Jersey" "Maine" and "Alaska." The loop can be easily modified to list these statistics for each state by switching it to a "for" loop that loops through each state in a complete list of states, then utilizing the functions BigSmallCounties and TotalAverageCountySize. I opted for the user-input approach because the output would be too long if each state were listed. 